In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import datetime
from pyspark.sql.functions import col, lit, avg
from pyspark.sql import *

In [2]:
#importing csv file from hdfs
user_data=user_log_data = sc.textFile("hdfs://localhost:54310/user/hive/warehouse/data/user_info.csv").map(lambda x:x.split(","))

In [3]:
user_df = sqlContext.createDataFrame(data=user_data.filter(lambda x:x[0]!='user_name'),
                             schema=user_data.filter(lambda x:x[0]=='user_name').collect()[0])

In [4]:
# creating new column named seconds with username
hours_list = []
for row in user_df.collect():
    # converting string type of datetime to datetime
    date_time = datetime.datetime.strptime(row['working_hours'], "%Y-%m-%d %H:%M:%S")
    seconds_data = date_time.hour * 3600 + date_time.minute *60 + date_time.second
    hours_list.append((row['user_name'], seconds_data))

In [5]:
 #creating new column dataframe to join with original dataframe( new_log_dataframe)
column = sqlContext.createDataFrame(hours_list, ('username',"seconds"))

In [6]:
#merge new column with the dataframe 
dataframe = (user_df
.join(column, col("user_name")==col("username")))


In [7]:
#obtaining avarage value of seconds by using aggregate function
average_data=dataframe.agg(avg(col("seconds")))
average = average_data.collect()[0][0]

In [8]:
#filtering dataframe with lowest average hours users

lowest_average_hours = dataframe.filter(dataframe['seconds'] < average)


In [9]:
lowest_average_hours.select("user_name","seconds").show()

+--------------------+-------+
|           user_name|seconds|
+--------------------+-------+
|       nikitapawar17|   6600|
|mail2anik.officia...|   5400|
|“shivnajalisangal...|    300|
|gaikwadr576@gmail...|  23400|
|adikumar2514@gmai...|  24599|
| dileep.bs@yahoo.com|  20100|
|er.mukulvij96@gma...|  24899|
|hakepratiksha55@g...|    300|
|vidyaramesh1428@g...|   9299|
|tekina.makin@gmai...|    600|
| addyp1911@gmail.com|  24000|
| blsonalib@gmail.com|  25500|
|20150773@dbatu.ac.in|  19500|
|ruchikachile30199...|  21899|
|shrutid2497@gmail...|  10200|
|vijaykumarbhavanu...|  25500|
|polelaxman001@gma...|   1200|
|ayush.saraf47@gma...|  21001|
|akshay.skawde@gma...|   3600|
|vishnu23kumar@gma...|  22200|
+--------------------+-------+
only showing top 20 rows



In [10]:
#filtering users having lowest average value
lowest_average_hours.select("user_name").show()

+--------------------+
|           user_name|
+--------------------+
|       nikitapawar17|
|mail2anik.officia...|
|“shivnajalisangal...|
|gaikwadr576@gmail...|
|adikumar2514@gmai...|
| dileep.bs@yahoo.com|
|er.mukulvij96@gma...|
|hakepratiksha55@g...|
|vidyaramesh1428@g...|
|tekina.makin@gmai...|
| addyp1911@gmail.com|
| blsonalib@gmail.com|
|20150773@dbatu.ac.in|
|ruchikachile30199...|
|shrutid2497@gmail...|
|vijaykumarbhavanu...|
|polelaxman001@gma...|
|ayush.saraf47@gma...|
|akshay.skawde@gma...|
|vishnu23kumar@gma...|
+--------------------+
only showing top 20 rows

